In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.optimize import minimize


X_train = np.load('initial_data/function_8/initial_inputs.npy')
Y_train = np.load('initial_data/function_8/initial_outputs.npy')
#0.231343-0.116511-0.192802-0.000000-1.000000-0.971517-0.109108-0.005687

X_train = np.append(X_train, [[0.231343,0.116511,0.192802,0.000000,1.000000,0.971517,0.109108,0.005687],
                              [0.050917,0.510974,0,0.51982,0.999999,0.964591,0.271744,0.828739],
                              [0.331151,0.212415,0.152484,0,0.999999,0.102819,0.394972,0.789628],
                              [0.,0.,0,0,0.999999,0.734351 ,0.999999,0.195222],
                              [0,0.445471,0,0,0,0.111183,0,0.925746],
                              [2.82290e-01,1.00000e-06,4.52278e-01,1.00000e-06,9.99999e-01,7.60695e-01
 ,1.17340e-01,9.91686e-01]], axis=0)
Y_train = np.append(Y_train, [9.6358729591536,9.4260415037804,9.6078496212371,8.512998485868101,9.2577071830184,9.4599831574609])

print(X_train)
print(Y_train)

[[6.04994453e-01 2.92215020e-01 9.08452748e-01 3.55506242e-01
  2.01668719e-01 5.75338005e-01 3.10310951e-01 7.34281377e-01]
 [1.78006959e-01 5.66222654e-01 9.94861845e-01 2.10325006e-01
  3.20152657e-01 7.07908792e-01 6.35384489e-01 1.07131627e-01]
 [9.07697668e-03 8.11626153e-01 5.20520360e-01 7.56866752e-02
  2.65111825e-01 9.16516894e-02 5.92415145e-01 3.67320262e-01]
 [5.06028164e-01 6.53730123e-01 3.63410779e-01 1.77981049e-01
  9.37283044e-02 1.97425331e-01 7.55826900e-01 2.92472339e-01]
 [3.59909264e-01 2.49075679e-01 4.95997170e-01 7.09214981e-01
  1.14987195e-01 2.89206921e-01 5.57295151e-01 5.93881726e-01]
 [7.78818344e-01 3.41949948e-03 3.37983130e-01 5.19527778e-01
  8.20906993e-01 5.37246689e-01 5.51347098e-01 6.60032086e-01]
 [9.08649322e-01 6.22496998e-02 2.38259546e-01 7.66603545e-01
  1.32335962e-01 9.90243814e-01 6.88067822e-01 7.42495941e-01]
 [5.86371444e-01 8.80735726e-01 7.45020752e-01 5.46034849e-01
  9.64887799e-03 7.48991763e-01 2.30907070e-01 9.79156228e-02]


In [2]:
# Define Gaussian Process
kernel = RBF(length_scale=np.ones(8)) + WhiteKernel(noise_level=1)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-5, normalize_y=True)
gp.fit(X_train, Y_train.ravel())

C:\Code\RoughWork\Jupyter\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Code\RoughWork\Jupyter\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(alpha=1e-05,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1),
                         normalize_y=True)

In [3]:
def ucb(X, gp, kappa=1.96):  # kappa can be adjusted based on your confidence level needs
    mean, std = gp.predict(X, return_std=True)
    return mean + kappa * std


In [4]:
def propose_point(gp, bounds=np.array([[0, 1]]*8), n_restarts=20):
    best_x = None
    best_ucb = float('-inf')

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, 8)):
        res = minimize(lambda x: -ucb(x.reshape(1, -1), gp), x0=starting_point, bounds=bounds, method='L-BFGS-B')
        if -res.fun > best_ucb:
            best_ucb = -res.fun
            best_x = res.x
            
    return best_x

new_point = propose_point(gp)
print("Proposed new hyperparameters:", new_point)

print(np.array2string(new_point,precision=6,separator='-',floatmode='fixed',formatter={'float':'{:0.6f}'.format}))


Proposed new hyperparameters: [0.22659543 0.05682445 0.19433813 0.41342435 1.         0.95750054
 0.25940035 0.59929966]
[0.226595-0.056824-0.194338-0.413424-1.000000-0.957501-0.259400-0.599300]
